This notebook loops through all the historical files and appends the airport to each row. The historical flights are mapped to separate csvs based on airline . Weather stations are also mapped to each row for easy lookup to the weather stations tables.

In [2]:
import pandas as pd
import os
import datetime
import numpy as np

In [3]:
errors = list()
airport_station = pd.read_csv("Airport_to_Weather_Stations.txt")
keys = airport_station["LocationID"].tolist()
values = airport_station['Nearest_Station'].tolist()
airport_to_station = {**dict(zip(keys, values)), **{'BKG': 'USC00230887', 'CLD': 'USW00003177', 'HDN': 'USC00053867', 'HHH': 'US1SCBF0052', 
                'MQT': 'US1MIMQ0016', 'AZA': 'USW00023104', 'ROP': 'CQC00914801', 'SPN': 'CQC00914855', 
                'UST': 'US1FLSJ0023', 'SCE': 'US1PACN0002', 'UTM': 'USC00229155', 'YAP': 'FMC00914429', 'YUM': 'USW00023195'}}

def parse_row(line, departure_airport, airline):
    row = line.replace('\"', '').replace("\n", "").split(",")
    #Field names
    #0 Carrier Code
    #1 Date (MM/DD/YYYY)
    #2 Flight Number
    #3 Tail Number
    #4 Destination Airport
    #5 Scheduled departure time
    #6 Actual departure time,
    #7 Scheduled elapsed time (Minutes)
    #8 Actual elapsed time (Minutes)
    #9 Departure delay (Minutes)

    date = row[1].split("/")
    year = date[2]
    if row[0] == airline and len(row[3]) > 1 and row[3] != "UNKNOW" and int(row[8]) > 0 and int(row[7]) > 0 and int(year) > 2004:
        #accept only if len(tailnumber) > 0 and tailnumber != "UNKNOW"
        #accept only if Actual time > 0
        #accept only if scheduled elapsed time > 0
        #carrier code, year, month, day, Flight#, Tail#, Source, Destination, departure_hour, depature_Minute, delay, station
        cc = row[0]
        #year
        month = date[0]
        day = date[1]
        flight = row[2]
        tail = row[3]
        source = departure_airport
        destination = row[4]
        departure = row[5].split(":")
        departure_hour = departure[0]
        departure_minute = departure[1]
        delay = row[9]
        try:
            station = airport_to_station[departure_airport]
        except:
            if departure_airport == "FCA":
                station = airport_to_station["GPI"]
            else:
                errors.append([cc, year, month, day, flight, tail, source, destination, departure_hour, departure_minute, delay])
                return None
        #map weather station to row.
        
        return [cc, year, month, day, flight, tail, source, destination, departure_hour, departure_minute, delay, station]
    
def parse_file(filename):
    airline = filename[0:2]
    departure_airport = filename[3:6]
    directory = 'csv\\' + filename
    f = open(directory, 'r')
    outputname = 'Processed_Flight_Data\\' + airline + '.txt'
    if os.path.exists(outputname):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not
    o = open(outputname,append_write)
    for line in f.readlines():
        if line[0] == '\"':
            output = parse_row(line, departure_airport, airline)
            if output != None:
                o.write(",".join(output) + '\n')
    f.close()
    o.close()
    pass

def main():
    for fn in os.listdir('csv\\'):
        filename = os.fsdecode(fn)
        if filename.endswith('.txt'):
            parse_file(filename)

#main()      #Don't want to accidentally run this code again...
print(errors)

[]


In this section we will find the departure of the previous flight with the same tail number and get it's departure delay.

In [34]:
columns = ["Airline", "Year", "Month", "Day", "Flight", "Tail", "Source", "Destination", "Departure_hour"
           , "Departure_minute", "Delay", "Station"]
def map_prev_flight(Airline):
    data = pd.read_csv("Processed_Flight_Data/" + Airline + ".txt", names = columns, header = None)
    data['Date'] = data.apply(lambda x: datetime.datetime(year=x.Year, month=x.Month, day=x.Day, hour = x.Departure_hour,
                                                          minute = x.Departure_minute), axis = 1)

    data = data.sort_values(['Tail', 'Date', 'Departure_hour'], ascending=[True, False, False])
    data['nTail'] = data['Tail'].shift(-1)
    data['nDate'] = data['Date'].shift(-1)
    data['nDelay'] = data['Delay'].shift(-1)

    data['Previous_Delay'] = np.where((data['nTail'] == data['Tail']) & 
                                      (data['Date'] - datetime.timedelta(hours = 12) < data['nDate']), data['nDelay'], 0)

    data.to_csv("Processed_Flight_Data/Mapped_with_Prev_Flight/" + Airline + ".csv", columns = columns + ['Previous_Delay'], 
                index = False, header = False)
    
for fn in os.listdir('Processed_Flight_Data\\'):
    filename = os.fsdecode(fn)
    if filename.endswith('.txt'):
        if filename[0:2] not in ['9E', 'AA', 'AQ', 'AS', 'B6', 'CO', 'DH', 'DL', 'EV', 'F9', 'FL', 'HA', 'HP']:
            map_prev_flight(filename[0:2])

Now we merge the flight data with the weather data from the Process Weather Data Notebook.

In [4]:
header = ["Airline", "Year", "Month", "Day", "Flight", "Tail", "Source", "Destination", "Departure_hour"
           , "Departure_minute", "Delay", "Station", "Previous_Delay"]
weather_data = pd.read_csv("Processed_Historical_Weather_Data.csv")
for fn in os.listdir('Processed_Flight_Data\\Mapped_with_Prev_Flight\\'):
    filename = os.fsdecode(fn)
    if filename.endswith('.csv'):
        flight_data = pd.read_csv("Processed_Flight_Data/Mapped_with_Prev_Flight/" + filename, header = None, names= header)
        flight_data['Date'] = flight_data.apply(lambda x: int(str(x.Year)+str(x.Month).zfill(2)+str(x.Day).zfill(2)), axis = 1)
        data = flight_data.merge(weather_data, left_on = ['Station', 'Date'], right_on = ['ID', 'Date'])
        data = data[data['TMAX'].notnull()]
        data = data[data['TMIN'].notnull()]
        data['SNOW'] = data['SNOW'].fillna(0)
        data['PRCP'] = data["PRCP"].fillna(0)
        data.to_csv("Processed_Flight_Data/Mapped_with_Prev_Flight/Final_Flight_Data/"
                    + filename,
                    columns = ["Airline", "Year", "Month", "Day"
                    ,"Tail", "Source", "Destination",  "Departure_hour", "Departure_minute"
                    , "Delay", "Station", "Previous_Delay", 'PRCP', 'SNOW', 'TMAX', 'TMIN']
                    ,index = False, header = False)

In [5]:
#Combine all  final flight data csvs to one file.
#Combine all csvs into 1 file
a = open("Final_historical_data.csv", 'a')
for fn in os.listdir('Processed_Flight_Data\\Mapped_with_Prev_Flight\\Final_Flight_Data\\'):
    filename = os.fsdecode(fn)
    if filename.endswith('.csv'):
        k = open('Processed_Flight_Data\\Mapped_with_Prev_Flight\\Final_Flight_Data\\' + filename, 'r')
        for line in k.readlines():
            a.write(line)
        k.close()
a.close()


In [7]:
#Get List of TailNumbers

names = ["Airline", "Year", "Month", "Day", "Flight", "Tail", "Source", "Destination", "Departure_hour"
           , "Departure_minute", "Delay", "Station"]

TailNumbers = set()
for fn in os.listdir('Processed_Flight_Data\\'):
    filename = os.fsdecode(fn)
    if filename.endswith('.txt'):
        data = pd.read_csv('Processed_Flight_Data/' + filename, header = None, names = names)
        TailNumbers.update(data['Tail'].unique())
            
f = open("Final_Tail_Number_List.txt", 'a')
for x in TailNumbers:
    f.write(x + '\n')
f.close()       